# Import JuMP and the solver GLPK

In [2]:
using JuMP, GLPK
import CSV
import DataFrames

using Suppressor

# Dataframes

In [ ]:
# Real-Time Price - DataFrame
priceRT_file = "Price-Real-Time-2023.csv"
priceRT_df = CSV.read(priceRT_file, DataFrames.DataFrame; drop=[:1])
priceRT_df = coalesce.(priceRT_df, 0)

In [ ]:
# Real-Time Generation - DataFrame
genRT_file = "Price-Real-Time-2023.csv"
genRT_df = CSV.read(genRT_file, DataFrames.DataFrame; drop=[:1])
genRT_df = coalesce.(genRT_df, 0)

# Problem

In [90]:
# 
priceDA = [0 10 20 30 40 50 100];
priceRT = priceRT_df
gRT     = genRT_df

G         = 100      # Maximum Generation/Quantity | Unit: MW
nScen     = 200      # Number os scenarios         | Unit: Scalar (One-Dimensional)

result = []
for ih in 1:7

    m = Model(GLPK.Optimizer)

    # qDA = Asset's Day-Ahead Generation Offered 
    @variable(m, 0 <= qDA[h in 1:24] <= G)
    @objective(m, Max, sum((priceDA[ih] * qDA[h] + 1/nScen * sum((priceRT[s,h] * (gRT[s,h] - qDA[h])) for s in 1:nScen)) for h in 1:24))

    optimize!(m)
    termination_status(m)
    # Printar modelo, equação de maximização e restrições
    # if ih == 1 
    #     println(m)
    #     println(solution_summary(m))
    # end
    push!(result, value(qDA[ih]))
end


# Results

In [95]:
# Create DataFrame
df = DataFrame(Price_DA = vec(priceDA), Offer_Gen_DA = result)

Row,Price_DA,Offer_Gen_DA
,Int64,Any
1,0,0.0
2,10,0.0
3,20,100.0
4,30,100.0
5,40,100.0
6,50,100.0
7,100,100.0
